链接到谷歌云盘

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
! pip install -qU \
    langchain==0.0.316 \
    openai==0.28.1  \
    tiktoken==0.5.1  \
    cohere \
    chromadb==0.4.15

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.8/479.8 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 74.5 MB/s eta 0:00:0

langchain：用途：LangChain是一个用于构建基于语言模型的应用程序的库。它可以帮助开发者快速构建和部署复杂的NLP工作流和生成任务，包括问答系统、聊天机器人等。
主要功能：
提供接口来连接不同的语言模型（如 OpenAI、Cohere）。
支持文本生成、问答、总结等多种任务。
提供工具来处理上下文管理和多步推理。
本demo用来提供OPENAI接口

tiktoken是一个高效的文本分词库，用于将文本转换为语言模型可以处理的token序列。
主要功能：
高效且快速地对文本进行分词。
支持多种语言模型的分词标准。
可用于优化模型输入和输出的处理速度

Cohere是一个提供语言模型服务的公司，其Python库用于与Cohere的API进行交互。
主要功能：
提供文本生成、分类、情感分析等功能。
支持自定义模型训练和部署。
允许开发者构建和优化自己的NLP应用程序。

ChromaDB是一个开源的数据库，用于存储和检索高维向量数据，常用于机器学习和推荐系统。
主要功能：
高效存储和检索嵌入向量。
提供快速的相似性搜索功能。
支持与各种机器学习框架集成，用于处理大型数据集和复杂查询。

In [3]:
import os
from langchain.chat_models import ChatOpenAI

os.environ["OPENAI_API_KEY"] = "my key"

chat = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"],
    model='gpt-3.5-turbo'
)

调用Chatgpt的API,使用Chatgpt为大语言模型

1.加载数据，以baichuan2论文为例

In [4]:
! pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 8.6 MB/s eta 0:00:00


In [5]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("https://arxiv.org/pdf/2309.10305.pdf")

pages = loader.load_and_split()

In [7]:
len(pages)

39

2.知识切片 将文档分割成均匀的块。每个块是一段原始文本


In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50,
)

docs = text_splitter.split_documents(pages)

In [9]:
len(docs)

215

3.利用embedding模型对每个文本片段进行向量化，并储存到向量数据库中

In [10]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma


embed_model = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents=docs, embedding=embed_model , collection_name="openai_embed")

In [11]:
vectorstore

4.通过向量相似度检索和问题最相关的K个文档。

In [34]:
query = "What's the main content of baichuan2?"
result = vectorstore.similarity_search(query ,k = 3)

In [35]:
len(result)

3

5.原始query与检索得到的文本组合起来输入到语言模型，得到最终的回答

In [36]:
def augment_prompt(query: str):
  # 获取top3的文本片段
  results = vectorstore.similarity_search(query, k=3)
  source_knowledge = "\n".join([x.page_content for x in results])#page_content为pdf文件提取内容格式
  # 构建prompt
  augmented_prompt = f"""Using the contexts below, answer the query.

  contexts:
  {source_knowledge}

  query: {query}"""
  return augmented_prompt

In [37]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content=""),
    HumanMessage(content=""),
    AIMessage(content=""),


]
# 创建prompt
prompt = HumanMessage(
    content=augment_prompt(query)
)

messages.append(prompt)

res = chat(messages)

print(res.content)

The main content of Baichuan 2 is its optimization for following human instructions, excelling at dialogue and context understanding. It also achieves significant improvements over Baichuan 1 on general benchmarks like MMLU, CMMLU, and C-Eval. Additionally, Baichuan 2 is specifically optimized to improve performance on math and code problems.
